In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# Data Processing

In [3]:
TRAIN_PATH = "../input/tensorflow-great-barrier-reef/train_images"
HEIGHT, WIDTH = 720, 1280
image_size = 640

vid0_path = "../input/tensorflow-great-barrier-reef/train_images/video_0"
vid1_path = "../input/tensorflow-great-barrier-reef/train_images/video_1"
vid2_path = "../input/tensorflow-great-barrier-reef/train_images/video_2"
vid_paths = [vid0_path, vid1_path, vid2_path]

vid0_ls = [os.path.join(vid0_path,f) for f in os.listdir(vid0_path)]
vid0_ls = sorted(vid0_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid1_ls = [os.path.join(vid1_path,f) for f in os.listdir(vid1_path)]
vid1_ls = sorted(vid1_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid2_ls = [os.path.join(vid2_path,f) for f in os.listdir(vid2_path)]
vid2_ls = sorted(vid2_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))
files_ls = [vid0_ls, vid1_ls, vid2_ls]

train_df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv",
                       sep = r',', skipinitialspace = True)

In [4]:
print(vid0_ls[:3])
print("\n",files_ls[0][:3])

In [5]:
len(files_ls[0])

In [6]:
train_df.head(5)

#### Helper functions

In [7]:
def get_oldpath(x):
    if x.video_id == 0:
        path = os.path.join(vid0_path,str(x.video_frame)+".jpg")
    elif x.video_id == 1:
        path = os.path.join(vid1_path,str(x.video_frame)+".jpg")
    else:
        path = os.path.join(vid2_path,str(x.video_frame)+".jpg")
        
    return path

def get_newpath(x):
    filename = f"{x.video_id}_{x.video_frame}.jpg"
    return os.path.join("./dataset", filename)

def get_filename(x):
    return f"{x.video_id}_{x.video_frame}.jpg"

In [8]:
import ast
from tqdm import tqdm
import shutil

nostar_df = train_df.copy() #[train_df.annotations == "[]"]

train_df = train_df[train_df.annotations != "[]"]
train_df["annotations"] = train_df["annotations"].map(lambda x : ast.literal_eval(x))

train_df["filepath"] = train_df.apply(lambda x : get_oldpath(x), axis=1)
train_df["newpath"] = train_df.apply(lambda x : get_newpath(x), axis=1)
train_df["filename"] = train_df.apply(lambda x : get_filename(x), axis=1)

os.makedirs("./dataset",exist_ok=True)

for i in tqdm(range(len(train_df))):
    src = train_df.iloc[i]["filepath"]
    dst = train_df.iloc[i]["newpath"]
    shutil.copy(src,dst)
    
train_df.head(3)

#### Main Dataframe

In [9]:
df = train_df
df = df.explode("annotations")

df["width"] = [WIDTH]*len(df)
df["height"] = [HEIGHT]*len(df)
df["label"] = ["starfish"]*len(df)

df["xmin"] = df.apply(lambda x : x.annotations["x"], axis=1)
df["ymin"] = df.apply(lambda x : x.annotations["y"], axis=1)
df["xmax"] = df.apply(lambda x : x.annotations["x"]+x.annotations["width"], axis=1)
df["ymax"] = df.apply(lambda x : x.annotations["y"]+x.annotations["height"], axis=1)

df.loc[df["xmax"] > 1280, "xmax"] = 1280
df.loc[df["ymax"] > 720, "ymax"] = 720

df = df.drop(["video_id","sequence","video_frame","sequence_frame","image_id","annotations","filepath","newpath"], axis=1)
df = df.reset_index(drop=True)
df.head(5)

In [10]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as npimg
import cv2

In [11]:
def draw_annotations(spot_fish_img, img_row):
    spot_fish_img = cv2.rectangle(spot_fish_img, (img_row['xmin'], img_row['ymin']),
              (img_row['xmax'], img_row['ymax']), (255,255,0), 2)
    return spot_fish_img

In [12]:
img_row = df.iloc[random.randint(0, len(df))]
img_row

In [13]:
original_img_path = os.path.join("./dataset", img_row['filename'])
original_img = npimg.imread(original_img_path)

fig, axs = plt.subplots(1, 1, figsize=(15, 10))
fig.tight_layout()

axs.imshow(original_img)
axs.set_title('Original Image')

In [14]:
spot_fish_img = original_img.copy()
draw_annotations(spot_fish_img, img_row)

fig, axs = plt.subplots(1, 1, figsize=(15, 10))
fig.tight_layout()

axs.imshow(spot_fish_img)
axs.set_title('spotfish Image')

# Data Modelling

Pretrained model

In [15]:
# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub


In [16]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

detector = hub.load(module_handle).signatures['default']


In [17]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def run_detector(detector, path):
    img = load_img(path)

    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    result = detector(converted_img)

    result = {key:value.numpy() for key,value in result.items()}

    print("Found %d objects." % len(result["detection_scores"]))

    #image_with_boxes = draw_boxes(
    #  img.numpy(), result["detection_boxes"],
    #  result["detection_class_entities"], result["detection_scores"])

    #display_image(image_with_boxes)
    return result

In [18]:
test_img_row = df.iloc[random.randint(0, len(df))]
test_img_path = os.path.join("./dataset", test_img_row['filename'])

result = run_detector(detector, test_img_path)


In [19]:
DETECTION_THRESHOLD = 0.1
MAX_IMG_NUM = 10
predictions = []
for index in range(len(result["detection_boxes"])):
    if index > MAX_IMG_NUM-1: break
    if result["detection_scores"][index] > DETECTION_THRESHOLD:
        #print(result["detection_scores"][index])
        predictions.append([result["detection_scores"][index], int(result["detection_boxes"][index][1] * WIDTH), 
                            int(result["detection_boxes"][index][0] * HEIGHT), 
                            int(result["detection_boxes"][index][3] * WIDTH), 
                            int(result["detection_boxes"][index][2] * HEIGHT)])

#print(predictions)

test_img = npimg.imread(test_img_path)
spot_fish_img = test_img.copy()
draw_annotations(spot_fish_img, img_row)

for x in predictions:
    spot_fish_img = cv2.rectangle(spot_fish_img, (x[1], x[2]), (x[3], x[4]),(255, 0, 0), 2)

fig, axs = plt.subplots(1, 1, figsize=(15, 10))
#fig.tight_layout()

axs.imshow(spot_fish_img)
axs.set_title('spotfish Image')

Classification

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils
from matplotlib import pyplot as plt
%matplotlib inline  
from keras import backend as K

In [21]:
df1 = df.drop(["width", "height","xmin","ymin","xmax","ymax"], axis=1)

df2 = nostar_df

df2["filename"] = df2.apply(lambda x : get_filename(x), axis=1)
df2["label"] = ["no_starfish"]*len(df2)

df2 = df2.drop(["video_id","sequence","video_frame","sequence_frame","image_id","annotations"], axis=1)


df_clas = pd.merge(df1, df2, on = ['filename', 'label'], how = 'outer')
df_clas = pd.DataFrame(df_clas, columns =['filename', 'label'])


#print(len(df1)+len(df2),len(df_clas))

# Evaluation

# Submission 